In [11]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df = pd.read_csv('Crimes_-_2001_to_Present.csv', low_memory=False)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9.0,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15.0,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631,6.0,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14.0,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15.0,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7784664 entries, 0 to 7784663
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              float64
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
dtypes: bool(2), float64(7), int64(3), object(1

In [6]:
df.isnull().sum()

ID                           0
Case Number                  4
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description     10381
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614848
Community Area          613476
FBI Code                     0
X Coordinate             86848
Y Coordinate             86848
Year                         0
Updated On                   0
Latitude                 86848
Longitude                86848
Location                 86848
dtype: int64

In [12]:
def clean_data(df):
    
    # Case Number - little nulls
    df = df.dropna(subset=['Case Number'])

    df = df.drop(columns=['X Coordinate','Y Coordinate','Updated On'])
    
    #Location
    df['Location Description'] = df['Location Description'].fillna("UNKNOWN")
    
    #The police District
    df['District'] = df['District'].fillna(df['District'].median())
    df['Ward'] = df['Ward'].fillna(df['Ward'].median())
    df['Community Area'] = df['Community Area'].fillna(df['Community Area'].median())
    
    ## 86848/ 7784664 = 0.001*100=1,1% of the dataset
    df = df.dropna(subset=['Latitude','Longitude','Location'])

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%m/%d/%Y %I:%M:%S %p')

    #if the year is the same from the year from date
    mismatch = df['Year'] != df['Date'].dt.year
    if mismatch.any():
        df = df[~mismatch]

    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Hour'] = df['Date'].dt.hour
    df['DayOfWeek'] = df['Date'].dt.dayofweek

    df['Working Hours'] = df['Hour'].apply(lambda x: 6 <= x < 18)
    df['Evening'] = df['Hour'].apply(lambda x: 18 <= x < 22)
    df['IsNight'] = df['Hour'].apply(lambda x: (x >= 22) or (x < 6))
    df['Weekend'] = df['DayOfWeek'].apply(lambda x: x >= 5)

    df['Month_Name'] = df['Date'].dt.strftime('%B')
    df['Day_of_Week_Name'] = df['Date'].dt.strftime('%A')

    df=df.drop_duplicates()

    #Not Year 2023
    df = df[df['Date'].dt.year!=2023]

    #Community Area - https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-Map/cauq-8yn6
    community_map = {
    0: "Unknown",
    1: "Rogers Park",
    2: "West Ridge",
    3: "Uptown",
    4: "Lincoln Square",
    5: "North Center",
    6: "Lake View",
    7: "Lincoln Park",
    8: "Near North Side",
    9: "Edison Park",
    10: "Norwood Park",
    11: "Jefferson Park",
    12: "Forest Glen",
    13: "North Park",
    14: "Albany Park",
    15: "Portage Park",
    16: "Irving Park",
    17: "Dunning",
    18: "Montclare",
    19: "Belmont Cragin",
    20: "Hermosa",
    21: "Avondale",
    22: "Logan Square",
    23: "Humboldt Park",
    24: "West Town",
    25: "Austin",
    26: "West Garfield Park",
    27: "East Garfield Park",
    28: "Near West Side",
    29: "North Lawndale",
    30: "South Lawndale",
    31: "Lower West Side",
    32: "Loop",
    33: "Near South Side",
    34: "Armour Square",
    35: "Douglas",
    36: "Oakland",
    37: "Fuller Park",
    38: "Grand Boulevard",
    39: "Kenwood",
    40: "Washington Park",
    41: "Hyde Park",
    42: "Woodlawn",
    43: "South Shore",
    44: "Chatham",
    45: "Avalon Park",
    46: "South Chicago",
    47: "Burnside",
    48: "Calumet Heights",
    49: "Roseland",
    50: "Pullman",
    51: "South Deering",
    52: "East Side",
    53: "West Pullman",
    54: "Riverdale",
    55: "Hegewisch",
    56: "Garfield Ridge",
    57: "Archer Heights",
    58: "Brighton Park",
    59: "McKinley Park",
    60: "Bridgeport",
    61: "New City",
    62: "West Elsdon",
    63: "Gage Park",
    64: "Clearing",
    65: "West Lawn",
    66: "Chicago Lawn",
    67: "West Englewood",
    68: "Englewood",
    69: "Greater Grand Crossing",
    70: "Ashburn",
    71: "Auburn Gresham",
    72: "Beverly",
    73: "Washington Heights",
    74: "Mount Greenwood",
    75: "Morgan Park",
    76: "O’Hare",
    77: "Edgewater"
    }

    df['Community Name'] = df['Community Area'].map(community_map)

    ##Seasonality
    df['Season'] = df['Month'].apply(lambda x: 'Summer' if x in [3,4,5,6,7,8] else 'Winter')

    #Map Location
    def map_location(location):
        loc = str(location).upper()
    
        if re.search(r'RESIDENCE|RESIDENTIAL|HOUSE|YARD|GARAGE|PORCH|ROOMING', loc):
            return 'RESIDENCE'
    
        elif re.search(r'VEHICLE|CAR|TRUCK|TAXI|BUS|CTA|DELIVERY', loc):
            return 'VEHICLE'
    
        elif re.search(r'SCHOOL|COLLEGE|UNIVERSITY', loc):
            return 'SCHOOL/UNIVERSITY'
    
        elif re.search(r'PARK|FOREST|LAKE|RIVER|PRAIRIE|STREET', loc):
            return 'PARK/OUTDOORS'
    
        elif re.search(r'STORE|RETAIL|GROCERY|APPLIANCE|DEPARTMENT|BANK|CLEANING|PAWN|MOVIE|THEATER|RESTAURANT|BAR|TAVERN|CLUB|HOTEL|MOTEL', loc):
            return 'COMMERCIAL/RETAIL'
    
        elif re.search(r'POLICE|GOVERNMENT|JAIL|COURT|HOSPITAL|MEDICAL|CLINIC|DENTAL|FIRE|STATION|ATM', loc):
            return 'GOVERNMENT/HOSPITAL'
    
        elif re.search(r'PARKING|GARAGE', loc):
            return 'PARKING'
    
        elif re.search(r'AIRPORT|TERMINAL|VENDING|ATS|AIRCRAFT', loc):
            return 'AIRPORT'
    
        elif re.search(r'CHA', loc):
            return 'CHA BUILDING / HOUSING'
    
        elif re.search(r'VESTIBULE|LOBBY|HALLWAY|STAIRWELL|GANGWAY', loc):
            return 'BUILDING INTERIORS'
    
        elif re.search(r'BOAT|WATERCRAFT', loc):
            return 'WATER TRANSPORT'
    
        else:
            return 'OTHER'

    df['Location Group'] = df['Location Description'].apply(map_location)

    return df

In [13]:
df = clean_data(df)

In [14]:
df.isnull().sum()

ID                      0
Case Number             0
Date                    0
Block                   0
IUCR                    0
Primary Type            0
Description             0
Location Description    0
Arrest                  0
Domestic                0
Beat                    0
District                0
Ward                    0
Community Area          0
FBI Code                0
Year                    0
Latitude                0
Longitude               0
Location                0
Month                   0
Day                     0
Hour                    0
DayOfWeek               0
Working Hours           0
Evening                 0
IsNight                 0
Weekend                 0
Month_Name              0
Day_of_Week_Name        0
Community Name          0
Season                  0
Location Group          0
dtype: int64

In [15]:
null_community = df[df['Community Name'].isna()]['Community Area'].unique()
print("Community Areas with null Community Name:", null_community)

Community Areas with null Community Name: []


In [16]:
df['Season'] = df['Month'].apply(lambda x: 'Summer' if x in [3,4,5,6,7,8] else 'Winter')

In [17]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Latitude,Longitude,Location,Month,Day,Hour,DayOfWeek,Working Hours,Evening,IsNight,Weekend,Month_Name,Day_of_Week_Name,Community Name,Season,Location Group
0,10224738,HY411648,2015-09-05 13:30:00,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9.0,12.0,61.0,08B,2015,41.815117,-87.670000,"(41.815117282, -87.669999562)",9,5,13,5,True,False,False,True,September,Saturday,New City,Winter,RESIDENCE
1,10224739,HY411615,2015-09-04 11:30:00,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15.0,29.0,25.0,06,2015,41.895080,-87.765400,"(41.895080471, -87.765400451)",9,4,11,4,True,False,False,False,September,Friday,Austin,Winter,VEHICLE
3,10224740,HY411595,2015-09-05 12:45:00,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14.0,35.0,21.0,18,2015,41.937406,-87.716650,"(41.937405765, -87.716649687)",9,5,12,5,True,False,False,True,September,Saturday,Avondale,Winter,OTHER
4,10224741,HY411610,2015-09-05 13:00:00,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15.0,28.0,25.0,08A,2015,41.881903,-87.755121,"(41.881903443, -87.755121152)",9,5,13,5,True,False,False,True,September,Saturday,Austin,Winter,OTHER
5,10224742,HY411435,2015-09-05 10:55:00,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,614,6.0,21.0,71.0,05,2015,41.744379,-87.658431,"(41.744378879, -87.658430635)",9,5,10,5,True,False,False,True,September,Saturday,Auburn Gresham,Winter,RESIDENCE


In [18]:
df['Primary Type'].unique()

array(['BATTERY', 'THEFT', 'NARCOTICS', 'ASSAULT', 'BURGLARY', 'ROBBERY',
       'OTHER OFFENSE', 'CRIMINAL DAMAGE', 'WEAPONS VIOLATION',
       'DECEPTIVE PRACTICE', 'CRIMINAL TRESPASS', 'MOTOR VEHICLE THEFT',
       'SEX OFFENSE', 'INTERFERENCE WITH PUBLIC OFFICER',
       'OFFENSE INVOLVING CHILDREN', 'PUBLIC PEACE VIOLATION',
       'PROSTITUTION', 'GAMBLING', 'CRIM SEXUAL ASSAULT',
       'LIQUOR LAW VIOLATION', 'ARSON', 'STALKING', 'KIDNAPPING',
       'INTIMIDATION', 'CONCEALED CARRY LICENSE VIOLATION',
       'NON - CRIMINAL', 'HUMAN TRAFFICKING', 'OBSCENITY',
       'CRIMINAL SEXUAL ASSAULT', 'PUBLIC INDECENCY',
       'OTHER NARCOTIC VIOLATION', 'NON-CRIMINAL', 'HOMICIDE',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'RITUALISM',
       'DOMESTIC VIOLENCE'], dtype=object)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7625080 entries, 0 to 7784663
Data columns (total 32 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ID                    int64         
 1   Case Number           object        
 2   Date                  datetime64[ns]
 3   Block                 object        
 4   IUCR                  object        
 5   Primary Type          object        
 6   Description           object        
 7   Location Description  object        
 8   Arrest                bool          
 9   Domestic              bool          
 10  Beat                  int64         
 11  District              float64       
 12  Ward                  float64       
 13  Community Area        float64       
 14  FBI Code              object        
 15  Year                  int64         
 16  Latitude              float64       
 17  Longitude             float64       
 18  Location              object        
 19  Month

In [20]:
df.shape

(7625080, 32)

In [21]:
df.describe(include='all')

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Latitude,Longitude,Location,Month,Day,Hour,DayOfWeek,Working Hours,Evening,IsNight,Weekend,Month_Name,Day_of_Week_Name,Community Name,Season,Location Group
count,7.625080e+06,7625080,7625080,7625080,7625080,7625080,7625080,7625080,7625080,7625080,7.625080e+06,7.625080e+06,7.625080e+06,7.625080e+06,7625080,7.625080e+06,7.625080e+06,7.625080e+06,7625080,7.625080e+06,7.625080e+06,7.625080e+06,7.625080e+06,7625080,7625080,7625080,7625080,7625080,7625080,7625080,7625080,7625080
unique,NaN,7624547,NaN,60923,402,36,541,216,2,2,NaN,NaN,NaN,NaN,26,NaN,NaN,NaN,888485,NaN,NaN,NaN,NaN,2,2,2,2,12,7,78,2,11
top,NaN,HZ140230,NaN,100XX W OHARE ST,0820,THEFT,SIMPLE,STREET,False,False,NaN,NaN,NaN,NaN,06,NaN,NaN,NaN,"(41.976290414, -87.905227221)",NaN,NaN,NaN,NaN,True,False,False,False,July,Friday,Loop,Summer,OTHER
freq,NaN,6,NaN,16046,620333,1606015,900301,1989927,5618650,6569380,NaN,NaN,NaN,NaN,1606015,NaN,NaN,NaN,14183,NaN,NaN,NaN,NaN,3906896,5927827,5604149,5494210,710539,1146295,777312,4010483,2150610
mean,6.957462e+06,NaN,2010-04-25 08:41:08.263022848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.186046e+03,1.129297e+01,2.275905e+01,3.707376e+01,NaN,2.009810e+03,4.184217e+01,-8.767152e+01,NaN,6.571261e+00,1.564410e+01,1.315676e+01,2.991901e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,6.340000e+02,NaN,2001-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,NaN,2.001000e+03,3.661945e+01,-9.168657e+01,NaN,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,3.758291e+06,NaN,2005-01-12 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.210000e+02,6.000000e+00,1.100000e+01,2.400000e+01,NaN,2.005000e+03,4.176872e+01,-8.771369e+01,NaN,4.000000e+00,8.000000e+00,9.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,6.960112e+06,NaN,2009-06-10 09:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.034000e+03,1.000000e+01,2.300000e+01,3.200000e+01,NaN,2.009000e+03,4.185589e+01,-8.766587e+01,NaN,7.000000e+00,1.600000e+01,1.400000e+01,3.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000259e+07,NaN,2015-03-21 15:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.731000e+03,1.700000e+01,3.200000e+01,5.300000e+01,NaN,2.015000e+03,4.190677e+01,-8.762820e+01,NaN,9.000000e+00,2.300000e+01,1.900000e+01,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.304315e+07,NaN,2022-12-31 23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,NaN,2.022000e+03,4.202291e+01,-8.752453e+01,NaN,1.200000e+01,3.100000e+01,2.300000e+01,6.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.to_csv("cleaned_data.csv", index=False)